In [1]:
from sklearn import datasets
X, y = datasets.load_svmlight_file("../Data/covtype.libsvm.binary.scale.bz2")

In [18]:
import zipfile
archive = zipfile.ZipFile('../Data/cox_regression.txt.zip')

In [19]:
AAA = archive.read("cox_regression.txt")

In [21]:
type(AAA)

bytes

In [41]:
import io
import zipfile

with zipfile.ZipFile('../Data/cox_regression.txt.zip') as zf:
    with io.TextIOWrapper(zf.open("cox_regression.txt"), encoding="utf-8") as f:
        a = pd.read_csv(f, sep = "\t", header = 0)

In [42]:
data = a.T

In [45]:
col_names = data.iloc[0,:]

In [46]:
data = data.iloc[1:,:]

In [47]:
data.columns = col_names

In [48]:
data = data.iloc

Hybridization REF,gene_id,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,...,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009,psiTPTE22|387590,tAKR|389932
TCGA-3Z-A93Z-01A-11R-A37O-07,normalized_count,0.0000,2.9608,11.4669,75.7771,563.9369,0.0000,118.5585,0.0000,0.0000,...,42.656,233.981,637.957,24.4644,646.74,3191.67,880.093,493.053,38.8922,0
TCGA-6D-AA2E-01A-11R-A37O-07,normalized_count,0.0000,2.4501,2.4069,49.6384,1313.5456,0.0000,155.9633,1.0793,0.0000,...,60.4425,239.072,1114.41,1.619,998.381,2715.06,1600.11,962.763,304.911,0
TCGA-A3-3306-01A-01R-0864-07,normalized_count,0.0000,3.2246,8.1625,70.2692,601.9669,0.0000,130.4348,1.5528,0.0000,...,118.012,457.039,624.224,1.0352,1190.99,2051.24,1098.34,987.578,15.0104,0
TCGA-A3-3307-01A-01R-0864-07,normalized_count,0.0000,1.8578,4.1055,94.6101,546.3303,0.0000,125.6881,1.8349,0.0000,...,89.4495,458.257,1166.06,10.5505,1184.4,3391.28,1043.12,734.404,39.4495,0
TCGA-A3-3308-01A-02R-1325-07,normalized_count,0.0000,6.4640,2.7113,89.7213,721.6424,0.0000,335.3596,0.9175,0.0000,...,91.7536,398.67,1576.79,0.9175,1245.56,4230.3,1719.92,918.913,12.3867,0


In [57]:
sorted(data.iloc[:,-1].unique())

[0.0,
 0.2329,
 0.2369,
 0.237,
 0.2498,
 0.2591,
 0.2619,
 0.2655,
 0.2743,
 0.2793,
 0.2817,
 0.29,
 0.2938,
 0.2945,
 0.3066,
 0.3103,
 0.3259,
 0.3279,
 0.3417,
 0.3433,
 0.3492,
 0.3532,
 0.3534,
 0.355,
 0.3614,
 0.3662,
 0.3674,
 0.3689,
 0.3772,
 0.3815,
 0.3828,
 0.3836,
 0.3937,
 0.4115,
 0.4136,
 0.4248,
 0.4268,
 0.4275,
 0.4284,
 0.4423,
 0.4534,
 0.4574,
 0.4608,
 0.4632,
 0.479,
 0.4943,
 0.496,
 0.4963,
 0.4995,
 0.5151,
 0.5155,
 0.5187,
 0.5291,
 0.5376,
 0.5501,
 0.5522,
 0.5645,
 0.6399,
 0.6704,
 0.6861,
 0.7744,
 0.834,
 0.8382,
 0.8518,
 0.8591,
 0.9891,
 1.0271,
 1.0731,
 1.0826,
 1.0956,
 1.2129,
 1.225,
 1.24,
 1.652,
 1.9481,
 2.2133,
 2.4272,
 2.5602,
 3.2498,
 4.8176,
 37.1638]